In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./model")

In [3]:
import torch
import argparse
import functools

In [4]:
import analyze
from quant import *
from outlier import *
from eval import *
from collections import defaultdict
from pprint import pprint
from modelutils_llama import quantize_model_llama, reorder_model_llama, quantize_model_gptq_llama,  add_act_quant_wrapper_llama
from modelutils_opt import quantize_model_opt, reorder_model_opt, quantize_model_gptq_opt,  add_act_quant_wrapper_opt
from modelutils_mixtral import quantize_model_mixtral, add_act_quant_wrapper_mixtral, reorder_model_mixtral
from parallel_utils import map_layers_to_multi_gpus
from LMClass import LMClass
from eval import pattern_match
from lm_eval import tasks as lm_tasks
from lm_eval import evaluator as lm_evaluator
from datautils import *
import qLinearLayer
from datasets import load_dataset
from transformers import LlamaTokenizer 

In [5]:
torch.set_printoptions(precision=10)

In [6]:
model = torch.load("./saved/llama2-7b_quantized.pth")

In [7]:
DEV = torch.device('cuda:1')

In [9]:
# traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')

tokenizer = LlamaTokenizer.from_pretrained("./llama2-7b", use_fast=False)
if tokenizer.bos_token_id != 1 or tokenizer.eos_token_id != 2:
    try:
        tokenizer.bos_token_id = 1
        tokenizer.eos_token_id = 2
        print(f"bos/eos tokens updated: {tokenizer.bos_token_id=},  {tokenizer.eos_token_id=}")
    except AttributeError:
        pass
        print(f"bos/eos tokens unchanged: {tokenizer.bos_token_id=},  {tokenizer.eos_token_id=}")
# trainenc = tokenizer("\n\n".join(traindata['text']), return_tensors='pt')
# testenc = tokenizer("\n\n".join(testdata['text']), return_tensors='pt')

In [12]:
input_data = tokenizer(testdata['text'][3], return_tensors='pt')
model=model.to(DEV)
model(input_data['input_ids'].to(DEV))

torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.float16
torch.

CausalLMOutputWithPast(loss=None, logits=tensor([[[-10.2421875000,  -5.1601562500,  -0.0357055664,  ...,
           -5.4960937500,  -6.1210937500,  -6.6171875000],
         [ -5.4726562500,   2.7246093750,   4.0234375000,  ...,
            6.2851562500,   1.6923828125,   4.5468750000],
         [-10.1171875000, -13.6328125000,   0.6201171875,  ...,
           -5.7773437500,  -4.3203125000,  -6.3632812500],
         ...,
         [ -9.0312500000, -12.8281250000,  10.7890625000,  ...,
           -4.0429687500,  -7.9570312500,  -5.5351562500],
         [ -8.4531250000,  -9.4062500000,   7.0820312500,  ...,
           -0.7255859375,  -5.2656250000,  -2.9628906250],
         [ -3.5957031250,  -2.2773437500,   1.6044921875,  ...,
           -3.1250000000,  -3.8789062500,  -2.5722656250]]], device='cuda:0',
       grad_fn=<ToCopyBackward0>), past_key_values=None, hidden_states=None, attentions=None)

# Test weight

In [9]:
# changed_layers = {}
for name, m in model.model.named_modules():
    if isinstance(m, qLinearLayer.QLinearLayer):
      weight = m.weight
      weight_scale = m.weight_scale.cpu()
      weight_int = torch.round(weight.to(torch.float32)/torch.repeat_interleave(weight_scale, 128, dim=-1))
      # weight_dequant = (weight_int * torch.repeat_interleave(weight_scale, 128, dim=-1)).to(torch.float16)
      weight_dequant = (weight_int.to(torch.float16) * torch.repeat_interleave(weight_scale.to(torch.float16), 128, dim=-1))
      err_cnt = ((weight-weight_dequant).abs() > 1e-3).sum()
      if(err_cnt > 0):
        print(name, err_cnt)
      # layer_v2 = qLinearLayer.QLinearLayerV2()
      # layer_v2.construct(m)
      # layer_v2 = layer_v2.to(DEV).to(torch.float32)

      # changed_layers[name] = layer_v2


# Test input

In [15]:
changed_layers = {}
for name, m in model.model.named_modules():
    if isinstance(m, qLinearLayer.QLinearLayer):
      layer_v2 = qLinearLayer.QLinearLayerV2()
      layer_v2.construct(m.to(torch.device('cpu')))
      layer_v2.name = name
      del m
      # changed_layers[name] = layer_v2.to(DEV)
      changed_layers[name] = layer_v2

for name, layer in changed_layers.items():
    analyze.set_nested_attr(model.model, name, layer.to(DEV))

In [16]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x QLlamaDecoderLayer(
        (self_attn): QLlamaAttention(
          (q_proj): QLinearLayerV2()
          (k_proj): QLinearLayerV2()
          (v_proj): QLinearLayerV2()
          (o_proj): QLinearLayerV2()
          (rotary_emb): LlamaRotaryEmbedding()
          (act_quant): Quantizer()
          (v_quant): Quantizer()
          (k_quant): Quantizer()
        )
        (mlp): QLlamaMLP(
          (gate_proj): QLinearLayerV2()
          (down_proj): QLinearLayerV2()
          (up_proj): QLinearLayerV2()
          (act_fn): SiLU()
          (act_quant): Quantizer()
        )
        (input_layernorm): QLlamaRMSNorm(
          (originalNorm): LlamaRMSNorm()
          (act_quant): Quantizer()
        )
        (post_attention_layernorm): QLlamaRMSNorm(
          (originalNorm): LlamaRMSNorm()
          (act_quant): Quantizer()
        )
      )
    )
    (norm): 

In [20]:
inputs = torch.load("./saved/llama2-7b_inputs.pth")

In [21]:
model_layer=model.model.layers
model_layer(input_data['input_ids'].to(DEV))

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [ ]:
input_scales = analyze.get_input_quant_param_dict(inputs)
weight_scales = analyze.get_weight_quant_param_dict(model)

# Test get scale functions

In [ ]:
test_input = inputs["layers.1.self_attn.q_proj.input"][0]
test_weight = model.model.layers[1].self_attn.q_proj.weight

In [ ]:
test_input_scale = analyze.get_input_qunat_scale(test_input)
print(test_input.shape)
print(test_input_scale.shape)

torch.Size([1, 2048, 4096])
torch.Size([2048, 32])


In [ ]:
test_weight_scale = analyze.get_weight_quant_scale(test_weight)
print(test_weight.shape)
print(test_weight_scale.shape)

torch.Size([4096, 4096])
torch.Size([4096, 32])


# Q proj example

In [65]:
input = inputs["layers.0.self_attn.q_proj.input"][0][0]
weight = model.model.layers[0].self_attn.q_proj.weight
input_scale = input_scales["layers.0.self_attn.q_proj.input.scale"][0]
weight_scale = weight_scales["layers.0.self_attn.q_proj.scale"]

In [51]:
ex_in_scale = torch.zeros_like(input)
for i in range(input_scale.shape[1]):
  for j in range(128):
    ex_in_scale[:, 128*i+j] = input_scale[:, i]

In [55]:
ex_weight_scale = torch.zeros_like(weight)
for i in range(weight_scale.shape[1]):
  for j in range(128):
    ex_weight_scale[:, 128*i+j] = weight_scale[:, i]

In [59]:
qin = torch.round(input / ex_in_scale)

In [60]:
qw = torch.round(weight / ex_weight_scale)

In [61]:
output = torch.zeros(2048, 4096)
for i in range(int(4096/128)):
  input = qin[:, 128*i:128*(i+1)]
  weight = qw[:, 128*i:128*(i+1)]
  scale_mat = torch.matmul(input_scale[:, i].reshape(-1, 1).float(), weight_scale[:, i].reshape(1, -1).float())
  out_psum = torch.matmul(input.float(), weight.T.float()) *scale_mat
  output += out_psum

In [62]:
outputs["layers.0.self_attn.q_proj.output"][0][0]

tensor([[ 0.1107788086, -0.4235839844,  0.7661132812,  ...,
         -0.1196899414,  0.6264648438,  0.1623535156],
        [-0.0408325195, -0.3610839844,  1.0117187500,  ...,
         -0.0342712402,  0.0709228516,  0.3554687500],
        [ 0.2474365234,  0.0408630371,  0.3076171875,  ...,
         -0.0485229492, -0.1354980469,  0.5034179688],
        ...,
        [ 0.0054931641, -0.4492187500,  1.0253906250,  ...,
         -0.0422668457,  0.0504150391,  0.2705078125],
        [ 0.2177734375, -1.2353515625,  2.0117187500,  ...,
         -0.1453857422,  0.4226074219, -0.6743164062],
        [ 0.1894531250, -0.3427734375,  0.9541015625,  ...,
         -0.0264587402,  0.0542907715,  0.2880859375]], dtype=torch.float16)

In [63]:
output

tensor([[ 0.1104393676, -0.4228451848,  0.7616682053,  ...,
         -0.1186680347,  0.6208702326,  0.1648930609],
        [-0.0407502465, -0.3595485091,  1.0115396976,  ...,
         -0.0342495590,  0.0719347894,  0.3554733098],
        [ 0.2474404275,  0.0442094766,  0.3077886701,  ...,
         -0.0486076251, -0.1366732419,  0.5032029152],
        ...,
        [ 0.0055385567, -0.4463545382,  1.0250202417,  ...,
         -0.0422924943,  0.0516813807,  0.2705228627],
        [ 0.2165240347, -1.2322372198,  2.0030839443,  ...,
         -0.1454128325,  0.4207982123, -0.6710996628],
        [ 0.1896059364, -0.3401398659,  0.9539331794,  ...,
         -0.0264508091,  0.0557389781,  0.2881198823]], grad_fn=<AddBackward0>)